# Script to get the location coordinates

## To-Do
- Destination 
1. Get the appropriate location name, port/airport.
2. Get the coordinates
- Source
1. Get the coordinates, if mode is AIR then it should be the airport of that city. If ocean, then the nearest port of that city. If road then that place coordinates

In [1]:
from geopy.geocoders import ArcGIS

import pandas as pd

In [58]:
nom = ArcGIS()

In [59]:
help(nom)

Help on ArcGIS in module geopy.geocoders.arcgis object:

class ArcGIS(geopy.geocoders.base.Geocoder)
 |  ArcGIS(username=None, password=None, *, referer=None, token_lifetime=60, scheme=None, timeout=DEFAULT_SENTINEL, proxies=DEFAULT_SENTINEL, user_agent=None, ssl_context=DEFAULT_SENTINEL, adapter_factory=None, auth_domain='www.arcgis.com', domain='geocode.arcgis.com')
 |  
 |  Geocoder using the ERSI ArcGIS API.
 |  
 |  Documentation at:
 |      https://developers.arcgis.com/rest/geocode/api-reference/overview-world-geocoding-service.htm
 |  
 |  Method resolution order:
 |      ArcGIS
 |      geopy.geocoders.base.Geocoder
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, username=None, password=None, *, referer=None, token_lifetime=60, scheme=None, timeout=DEFAULT_SENTINEL, proxies=DEFAULT_SENTINEL, user_agent=None, ssl_context=DEFAULT_SENTINEL, adapter_factory=None, auth_domain='www.arcgis.com', domain='geocode.arcgis.com')
 |      :param str username: A

In [7]:
location = nom.geocode('Aurobindo Unit III, India')

In [9]:
location.address

'Aurobindo Unit-XII'

In [58]:
print(str(location.latitude) + ',' + str(location.longitude))

17.52389000000005,78.34978000000007


In [61]:
country = nom.reverse('17.52389000000005,78.34978000000007')

In [62]:
help(country)

Help on Location in module geopy.location object:

class Location(builtins.object)
 |  Location(address, point, raw)
 |  
 |  Contains a parsed geocoder response. Can be iterated over as
 |  ``(location<String>, (latitude<float>, longitude<Float))``.
 |  Or one can access the properties ``address``, ``latitude``,
 |  ``longitude``, or ``raw``. The last
 |  is a dictionary of the geocoder's response for this item.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getitem__(self, index)
 |      Backwards compatibility with geopy<0.98 tuples.
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, address, point, raw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  __ne__(self, other)
 |      Return self!=value.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setstate__(self, state)
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  -------------

In [2]:
from geopy.geocoders import TomTom

nom = TomTom()

nom.geocode('ABBVIE (Abbott) Logis. UK')

TypeError: __init__() missing 1 required positional argument: 'api_key'

In [3]:
raw_airport_db = pd.read_csv('Data/airports.csv')

In [36]:
only_airports = raw_airport_db[raw_airport_db['type'].str.lower().str.contains('airport')]

In [43]:
only_airports.iso_country.isnull().sum()

255

In [55]:
from geopy.geocoders import Nominatim

# Create a Nominatim geolocator object
geolocator = Nominatim(user_agent='user_agent')

# Get location information for a place name
location = geolocator.reverse('15.359210,73.939670')

# Print the latitude, longitude, and country name
print(help(location))
# print(location.)


Help on Location in module geopy.location object:

class Location(builtins.object)
 |  Location(address, point, raw)
 |  
 |  Contains a parsed geocoder response. Can be iterated over as
 |  ``(location<String>, (latitude<float>, longitude<Float))``.
 |  Or one can access the properties ``address``, ``latitude``,
 |  ``longitude``, or ``raw``. The last
 |  is a dictionary of the geocoder's response for this item.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getitem__(self, index)
 |      Backwards compatibility with geopy<0.98 tuples.
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, address, point, raw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  __ne__(self, other)
 |      Return self!=value.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setstate__(self, state)
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  -------------